# Total points 8

# In  the homework
- We'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://developer.twitter.com/en  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [1]:
client_key='gktTexWbbaMGUFs9bvbSvyZRb'
client_secret='UjEvqXcReCQKdn33oOqZu7lsXOtMoHBtQjlWWaFZP3ev1jI9tu'
# bearer token
bearer_token='AAAAAAAAAAAAAAAAAAAAAO8ZhgEAAAAA6C%2BjlPmqoEYZIh3D7UPNpUWUN4k%3DS6PtQPDHwVKzHyemkqxK1FQRgSKo7Ewk6BoxOVnKiaeMtbMQ34'

In [2]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [3]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [4]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [5]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAO8ZhgEAAAAA6C%2BjlPmqoEYZIh3D7UPNpUWUN4k%3DS6PtQPDHwVKzHyemkqxK1FQRgSKo7Ewk6BoxOVnKiaeMtbMQ34'}

In [6]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [37]:
# Below tw_limit refers to the max tweets we can retrieve from a single
# request.  I'm setting it to 10 for simplicity.  Max_tweets represents
# the total number of tweets we'd like to retrieve.  We can, of course, 
# retrieve 30 tweets with a single call, but we're going to make three
# requests with max set to 10 in order to accumulate 30 tweets total.

max_tweets=30
tw_limit=10# we can get only 100 tweet per call with standard search api
tweets = []
users =[]

In [38]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'query': 'climate change',
                'max_results': 10,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}

search_url = base_url+'2/tweets/search/recent'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [39]:
climate_tweets = response.json()


In [40]:
len(climate_tweets['data'])

10

In [41]:
tweets +=climate_tweets['data']


In [42]:
ids = [tw['id'] for tw in climate_tweets['data']]    

In [43]:
min_id = min(ids)

In [44]:
tw_ids = []
tw_ids+=ids

In [45]:
users.extend(climate_tweets['includes']['users'])

As per twitter api documentation we can use

until_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [46]:
 parameters = { 'query': 'climate change',
                'max_results': tw_limit,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
                'until_id': min_id
              }

for  i in range(max_tweets//tw_limit -1):
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'2/tweets/search/recent'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['data']]
    tw_ids+=ids
    print(i, len(climate_tweets['data']))
    min_id = min(ids)
    tweets +=climate_tweets['data']
    users.extend(climate_tweets['includes']['users'])
    if 'next_token' in response.json()['meta']:
					parameters['next_token'] = response.json()['meta']['next_token']

searching tweets with id  < 1579610183688286208
0 10
searching tweets with id  < 1579610079388905475
1 10


In [47]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 30


# Let's print some tweets

In [48]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [49]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'author_id': '838601462766784514',
  'conversation_id': '1579610252450082826',
  'created_at': '2022-10-10T23:10:06.000Z',
  'edit_history_tweet_ids': ['1579610252450082826'],
  'entities': {'annotations': [{'end': 63,
                                'normalized_text': 'Europe',
                                'probability': 0.9894,
                                'start': 58,
                                'type': 'Place'},
                               {'end': 100,
                                'normalized_text': 'Poland',
                                'probability': 0.9962,
                                'start': 95,
                                'type': 'Place'}],
               'mentions': [{'end': 16,
                             'id': '707231479047315456',
                             'start': 3,
                             'username': 'PeterSweden7'}]},
  'id': '1579610252450082826',
  'lang': 'en',
  'public_metrics': {'like_count': 0,
                     'quote_c

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [51]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

RT @PeterSweden7: The climate change fanatics have caused Europe to have a energy crisis. 

In Poland they are now burning GARBAGE to keep…

According to recent warnings from experts, climate change may cause potatoes to no longer grow in Scottish soil over the next several years. In certain areas of the country, farmers are dealing with the worst drought they have seen in years.
https://t.co/Du8lPFzzQ9 https://t.co/QEPcJSkt1X

RT @Heykki80: Viime vuonna julkaistussa tutkimuksessa arvioitiin maailmalla kuolevan 5 miljoonan ihmisen per vuosi ilmastonmuutoksesta joht…

RT @PeterSweden7: The climate change fanatics have caused Europe to have a energy crisis. 

In Poland they are now burning GARBAGE to keep…



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [52]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['lang', 'created_at', 'text', 'source', 'public_metrics', 'entities', 'id', 'conversation_id', 'author_id', 'reply_settings', 'edit_history_tweet_ids'])
11


In [53]:
import pandas as pd
import copy

# Generators for building DataFrames

# Q2 (2 point) Complete following   python generator *tweet_generator*. Generator yield modified tweet .
For each tweet in tweets
- Replace **author_id** attribute with **user_id**. Note that user_id isn't a returned field. We're just associating the new key user_id with the value associated with author_id.
- Replace **referenced_tweets** attribute with **referenced_tweets_ids** and value(s) with **tweet['referenced_tweets']['id']** value(s). Same thing as above. We're creating a new key and assigning to it a value that already exists in our data.
- Keep other attributes in the tweet as such.

In [54]:
def tweet_generator(tweets):
    for tweet in tweets:
        try:
            modified_tweet = copy.deepcopy(tweet)
            modified_tweet['user_id'] = modified_tweet.pop('author_id')
            modified_tweet['referenced_tweets_ids'] = modified_tweet['referenced_tweets'][0]['id']
            modified_tweet['retweet_count'] = modified_tweet['public_metrics']['retweet_count']
            yield modified_tweet     
        except KeyError as e:
            continue

In [55]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'lang': 'en',
  'created_at': '2022-10-10T23:10:06.000Z',
  'text': 'RT @PeterSweden7: The climate change fanatics have caused Europe to have a energy crisis. \n\nIn Poland they are now burning GARBAGE to keep…',
  'source': 'Twitter for iPhone',
  'public_metrics': {'retweet_count': 1343,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'entities': {'mentions': [{'start': 3,
     'end': 16,
     'username': 'PeterSweden7',
     'id': '707231479047315456'}],
   'annotations': [{'start': 58,
     'end': 63,
     'probability': 0.9894,
     'type': 'Place',
     'normalized_text': 'Europe'},
    {'start': 95,
     'end': 100,
     'probability': 0.9962,
     'type': 'Place',
     'normalized_text': 'Poland'}]},
  'id': '1579610252450082826',
  'conversation_id': '1579610252450082826',
  'referenced_tweets': [{'type': 'retweeted', 'id': '1579516613115924481'}],
  'reply_settings': 'everyone',
  'edit_history_tweet_ids': ['1579610252450082826'],
  'user_id': '838601462766

In [56]:
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [57]:
climate_feed_df

,lang,created_at,text,source,public_metrics,entities,id,conversation_id,referenced_tweets,reply_settings,edit_history_tweet_ids,user_id,referenced_tweets_ids,retweet_count,in_reply_to_user_id,withheld
0,en,2022-10-10T23:10:06.000Z,RT @PeterSweden7: The climate change fanatics ...,Twitter for iPhone,"{'retweet_count': 1343, 'reply_count': 0, 'lik...","{'mentions': [{'start': 3, 'end': 16, 'usernam...",1579610252450082826,1579610252450082826,"[{'type': 'retweeted', 'id': '1579516613115924...",everyone,[1579610252450082826],838601462766784514,1579516613115924481,1343,NaN,NaN
1,fi,2022-10-10T23:10:00.000Z,RT @Heykki80: Viime vuonna julkaistussa tutkim...,Twitter for Android,"{'retweet_count': 4, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 3, 'end': 12, 'usernam...",1579610228362194957,1579610228362194957,"[{'type': 'retweeted', 'id': '1579390244109312...",everyone,[1579610228362194957],2405097439,1579390244109312000,4,NaN,NaN
2,en,2022-10-10T23:10:00.000Z,RT @PeterSweden7: The climate change fanatics ...,Twitter for iPhone,"{'retweet_count': 1343, 'reply_count': 0, 'lik...","{'mentions': [{'start': 3, 'end': 16, 'usernam...",1579610224981606400,1579610224981606400,"[{'type': 'retweeted', 'id': '1579516613115924...",everyone,[1579610224981606400],475408301,1579516613115924481,1343,NaN,NaN
3,en,2022-10-10T23:09:59.000Z,RT @CurtisHebert: The insanity of a man who is...,Twitter for iPhone,"{'retweet_count': 177, 'reply_count': 0, 'like...","{'mentions': [{'start': 3, 'end': 16, 'usernam...",1579610224167890952,1579610224167890952,"[{'type': 'retweeted', 'id': '1579478339232071...",everyone,[1579610224167890952],53231656,1579478339232071684,177,NaN,NaN
4,en,2022-10-10T23:09:57.000Z,RT @catherine___c: The cost of living is soari...,Twitter for Android,"{'retweet_count': 32, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 17, 'usernam...",1579610216169353219,1579610216169353219,"[{'type': 'retweeted', 'id': '1579586274935336...",everyone,[1579610216169353219],904219877702905856,1579586274935336961,32,NaN,NaN
5,en,2022-10-10T23:09:56.000Z,RT @mazemoore: Everything you need to know abo...,Twitter for Android,"{'retweet_count': 576, 'reply_count': 0, 'like...","{'mentions': [{'start': 3, 'end': 13, 'usernam...",1579610210813227013,1579610210813227013,"[{'type': 'retweeted', 'id': '1579548624090198...",everyone,[1579610210813227013],1519983442146230272,1579548624090198016,576,NaN,NaN
6,en,2022-10-10T23:09:52.000Z,"RT @OccupyDemocrats: BREAKING: Over 60,000 sal...",Twitter for iPhone,"{'retweet_count': 207, 'reply_count': 0, 'like...","{'mentions': [{'start': 3, 'end': 19, 'usernam...",1579610194975535104,1579610194975535104,"[{'type': 'retweeted', 'id': '1579598666683527...",everyone,[1579610194975535104],752708580,1579598666683527170,207,NaN,NaN
7,en,2022-10-10T23:09:50.000Z,@DoBetterLaura This machine fights deathly pol...,Twitter for iPhone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 0, 'end': 14, 'usernam...",1579610186490458115,1579574684185923584,"[{'type': 'replied_to', 'id': '157957468418592...",everyone,[1579610186490458115],1437181362717474819,1579574684185923584,0,1177225666317692929,NaN
8,en,2022-10-10T23:09:50.000Z,RT @ZeitgeistPOL: Heather du Plessis-Allan: “T...,Twitter for Android,"{'retweet_count': 26, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 16, 'usernam...",1579610183688286208,1579610183688286208,"[{'type': 'retweeted', 'id': '1535000534310039...",everyone,[1579610183688286208],335853669,1535000534310039553,26,NaN,NaN
9,en,2022-10-10T23:09:49.000Z,RT @Martyupnorth_2: I wish I had more time to ...,Twitter for Android,"{'retweet_count': 333, 'reply_count': 0, 'like...","{'mentions': [{'start': 3, 'end': 18, 'usernam...",1579610182203879431,1579610182203879431,"[{'type': 'retweeted', 'id': '1579474736572399...",everyone,[1579610182203879431],271590345,1579474736572399616,333,NaN,NaN


# Q3(1 points) List datatype using climate_feed_df. Describe which attributes doesn't have correct data types. What should be correct datatype?

In [27]:
# write code here datatype listing
climate_feed_df.dtypes

referenced_tweets         object
created_at                object
reply_settings            object
text                      object
entities                  object
conversation_id           object
lang                      object
edit_history_tweet_ids    object
id                        object
public_metrics            object
source                    object
user_id                   object
referenced_tweets_ids     object
retweet_count              int64
geo                       object
in_reply_to_user_id       object
dtype: object

# Write columns with wrong datatype and mention correct data type too.

- reply_settings:            &nbsp;**string**
- lang:                      &nbsp;**string**
- source:                    &nbsp;**string**
- referenced_tweets:         &nbsp;**dict**
- entities:                  &nbsp;**dict**
- conversation_id:           &nbsp;**string or int**
- id:                        &nbsp;**string or int**
- public_metrics:            &nbsp;**dict**
- text:                      &nbsp;**string**
- edit_history_tweet_ids:    &nbsp;**list** (could be converted to string or int)
- created_at:                &nbsp;**datetime**
- user_id:                   &nbsp;**string or int**
- referenced_tweets_ids:     &nbsp;**string or int**
- retweet_count:             &nbsp;**int64**
- in_reply_to_user_id:       &nbsp;**string or int**


# Q4 (1.5 point) Complete following tweet_user_generator. Generator should yield only *user* value from a tweet

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [58]:
def tweet_user_generator(tweets):
    for user in tweets:
        try:
            yield user
        except KeyError:
            continue

In [59]:
user_df = pd.DataFrame(tweet_user_generator(climate_tweets))

In [60]:
user_df

,0
0,data
1,includes
2,meta


# Q5(1 point) List datatypes of columns using user_df. Describe which attributes doesn't have correct data types. What should be correct datatype for each case?

In [31]:
user_df.dtypes

0    object
dtype: object

# write columns with wrong datatype and mention correct data type too.

The column 0 lists all the keys to dicts-- this should be a string.

# Q6 (1 point) Complete following retweeted_status_generator. Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary.

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [32]:
retweeted_status_df = climate_feed_df[['id', 'retweet_count']]

In [33]:
retweeted_status_df.head()

,id,retweet_count
0,1579608055800803330,4
1,1579608051510030337,0
2,1579608050155257856,864
3,1579608049706471424,0
4,1579608022741299201,91


# Q7 (1 point) Write code to find number of rows in retweeted_status_df that were not retweeted. print number of such rows.

In [34]:
(retweeted_status_df['retweet_count'] == 0).sum()

6